# Imports

In [ ]:
import openslide as op
from PIL import Image
import numpy as np
import random
import glob
import os
import util
import h5py
import cv2
from skimage import measure
from matplotlib import pyplot as plt
from scipy import misc, ndimage
from skimage import morphology
from skimage import color
from skimage import io
import time
from extract import extractFiles
import configparser
import ast
from util import extend_glob

# Configuration

In [ ]:
config = configparser.RawConfigParser(interpolation=configparser.ExtendedInterpolation())
config.read('cytonet.cfg')
section = 'extraction'

In [ ]:
filenames=  ast.literal_eval(config.get(section, 'filenames'))
output_folder = config.get(section, 'output_folder')
mask_pattern= config.get(section, 'mask_pattern')

classes = ast.literal_eval(config.get(section, 'classes') if config.has_option(section, 'classes') else config.get('general', 'classes'))
load_level=config.getint(section, 'load_level') if config.has_option(section, 'patch_size') else config.getint('general', 'load_level')
patch_size = config.getint(section, 'patch_size') if config.has_option(section, 'patch_size') else config.getint('general', 'patch_size')

# Code Execution

In [ ]:
# Get the files for training
files_train= extend_glob(filenames)
print(files_train)

In [ ]:
# Training extraction
import time
start = time.time()
extractFiles(files_train,
            output_folder, 
            mask_pattern, 
            classes, 
            load_level, 
            patch_size)
end = time.time()
print("Time : ", end-start)